In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/face-images1/Final Testing Images/face14/1face14.jpg
/kaggle/input/face-images1/Final Testing Images/face14/2face14.jpg
/kaggle/input/face-images1/Final Testing Images/face14/3face14.jpg
/kaggle/input/face-images1/Final Testing Images/face14/4face14.jpg
/kaggle/input/face-images1/Final Testing Images/face4/2face4.jpg
/kaggle/input/face-images1/Final Testing Images/face4/1face4.jpg
/kaggle/input/face-images1/Final Testing Images/face4/3face4.jpg
/kaggle/input/face-images1/Final Testing Images/face4/4face4.jpg
/kaggle/input/face-images1/Final Testing Images/face15/1face15.jpg
/kaggle/input/face-images1/Final Testing Images/face15/4face15.jpg
/kaggle/input/face-images1/Final Testing Images/face15/3face15.jpg
/kaggle/input/face-images1/Final Testing Images/face15/2face15.jpg
/kaggle/input/face-images1/Final Testing Images/face16/2face16.jpg
/kaggle/input/face-images1/Final Testing Images/face16/3face16.jpg
/kaggle/input/face-images1/Final Testing Images/face16/4face16.jpg
/ka

In [2]:
TrainingImagePath='/kaggle/input/face-images1/Final Training Images'

import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout
import matplotlib.pyplot as plt

train_datagen = ImageDataGenerator(
    shear_range=0.1,
    zoom_range=0.1,
    horizontal_flip=True,
    rescale=1.0/255.0
)
 
test_datagen = ImageDataGenerator(rescale=1.0/255.0)

training_set = train_datagen.flow_from_directory(
        TrainingImagePath,
        target_size=(64, 64),
        batch_size=32,
        class_mode='categorical')
 
test_set = test_datagen.flow_from_directory(
        TrainingImagePath,
        target_size=(64, 64),
        batch_size=32,
        class_mode='categorical')

test_set.class_indices

2024-05-26 10:56:33.485889: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-05-26 10:56:33.486061: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-05-26 10:56:33.644344: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


Found 244 images belonging to 16 classes.
Found 244 images belonging to 16 classes.


{'face1': 0,
 'face10': 1,
 'face11': 2,
 'face12': 3,
 'face13': 4,
 'face14': 5,
 'face15': 6,
 'face16': 7,
 'face2': 8,
 'face3': 9,
 'face4': 10,
 'face5': 11,
 'face6': 12,
 'face7': 13,
 'face8': 14,
 'face9': 15}

In [3]:

TrainClasses=training_set.class_indices

ResultMap={}
for faceValue,faceName in zip(TrainClasses.values(),TrainClasses.keys()):
    ResultMap[faceValue]=faceName

import pickle
with open("ResultsMap.pkl", 'wb') as fileWriteStream:
    pickle.dump(ResultMap, fileWriteStream)
 
print("Mapping of Face and its ID",ResultMap)

OutputNeurons=len(ResultMap)
print('\n The Number of output neurons: ', OutputNeurons)

Mapping of Face and its ID {0: 'face1', 1: 'face10', 2: 'face11', 3: 'face12', 4: 'face13', 5: 'face14', 6: 'face15', 7: 'face16', 8: 'face2', 9: 'face3', 10: 'face4', 11: 'face5', 12: 'face6', 13: 'face7', 14: 'face8', 15: 'face9'}

 The Number of output neurons:  16


In [4]:

from keras.models import Sequential
from keras.layers import Convolution2D
from keras.layers import MaxPool2D
from keras.layers import Flatten
from keras.layers import Dense
from keras.layers import Input 

classifier= Sequential()

classifier.add(Input(shape=(64, 64, 3)))

classifier.add(Convolution2D(32, kernel_size=(5, 5), strides=(1, 1), input_shape=(64,64,3), activation='relu'))

classifier.add(MaxPool2D(pool_size=(2,2)))

classifier.add(Convolution2D(64, kernel_size=(5, 5), strides=(1, 1), activation='relu'))
 
classifier.add(MaxPool2D(pool_size=(2,2)))

classifier.add(Flatten())

classifier.add(Dense(64, activation='relu'))
 
classifier.add(Dense(OutputNeurons, activation='softmax'))

classifier.compile(loss='categorical_crossentropy', optimizer = 'adam', metrics=["accuracy"])

import time
StartTime=time.time()

classifier.fit(
                    training_set,
                    steps_per_epoch=30,
                    epochs=10,
                    validation_data=test_set,
                    validation_steps=10)
 
EndTime=time.time()
print("###### Total Time Taken: ", round((EndTime-StartTime)/60), 'Minutes ######')

/opt/conda/lib/python3.10/site-packages/keras/src/layers/convolutional/base_conv.py:99: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(


Epoch 1/10


/opt/conda/lib/python3.10/site-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:120: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


 8/30 ━━━━━━━━━━━━━━━━━━━━ 3s 150ms/step - accuracy: 0.0651 - loss: 2.8263

/opt/conda/lib/python3.10/contextlib.py:153: UserWarning: Your input ran out of data; interrupting training. Make sure that your dataset or generator can generate at least `steps_per_epoch * epochs` batches. You may need to use the `.repeat()` function when building your dataset.
  self.gen.throw(typ, value, traceback)


30/30 ━━━━━━━━━━━━━━━━━━━━ 6s 104ms/step - accuracy: 0.0534 - loss: 2.8143 - val_accuracy: 0.0697 - val_loss: 2.7620
Epoch 2/10
30/30 ━━━━━━━━━━━━━━━━━━━━ 3s 80ms/step - accuracy: 0.0418 - loss: 2.7626 - val_accuracy: 0.1270 - val_loss: 2.7455
Epoch 3/10
30/30 ━━━━━━━━━━━━━━━━━━━━ 3s 79ms/step - accuracy: 0.0723 - loss: 2.7383 - val_accuracy: 0.1598 - val_loss: 2.6832
Epoch 4/10
30/30 ━━━━━━━━━━━━━━━━━━━━ 5s 79ms/step - accuracy: 0.0983 - loss: 2.6709 - val_accuracy: 0.0943 - val_loss: 2.5611
Epoch 5/10
30/30 ━━━━━━━━━━━━━━━━━━━━ 5s 80ms/step - accuracy: 0.1174 - loss: 2.5530 - val_accuracy: 0.2090 - val_loss: 2.3417
Epoch 6/10
30/30 ━━━━━━━━━━━━━━━━━━━━ 5s 79ms/step - accuracy: 0.2212 - loss: 2.3296 - val_accuracy: 0.3443 - val_loss: 2.0869
Epoch 7/10
30/30 ━━━━━━━━━━━━━━━━━━━━ 5s 78ms/step - accuracy: 0.3262 - loss: 2.1219 - val_accuracy: 0.3811 - val_loss: 1.8740
Epoch 8/10
30/30 ━━━━━━━━━━━━━━━━━━━━ 5s 61ms/step - accuracy: 0.5463 - loss: 1.7827 - val_accuracy: 0.6639 - val_loss: 1

In [5]:
import numpy as np
from keras.preprocessing import image
 
ImagePath='/kaggle/input/face-images1/Final Training Images/face13/image_0288_Face_1.jpg'
test_image=image.load_img(ImagePath,target_size=(64, 64))
test_image=image.img_to_array(test_image)
 
test_image=np.expand_dims(test_image,axis=0)
 
result=classifier.predict(test_image,verbose=0)
 
print('####'*10)
print('Prediction is: ',ResultMap[np.argmax(result)])

########################################
Prediction is:  face11
